Генерация ИИ обложек для видео

In [ ]:
!pip install torch
!pip install torchaudio
!pip install torch diffusers
!pip install torchvision
!pip install transformers
!pip install accelerate
!pip install translate
!pip install rutube-downloader m3u8 alive_progress
!pip install opencv-python
!pip install beautifulsoup4
!pip install detectron2
!pip install flask-ngrok
!git clone https://github.com/artemgoncarov/sochi-web
!pip install "git+https://github.com/ai-forever/Kandinsky-2.git"
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install transformers

In [ ]:
!python -m pip install pyyaml
import sys, os, distutils.core
!git clone "https://github.com/facebookresearch/detectron2"
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i "s/OPENCV=0/OPENCV=1/" Makefile
!sed -i "s/GPU=0/GPU=1/" Makefile
!sed -i "s/CUDNN=0/CUDNN=1/" Makefile
!sed -i "s/CUDNN_HALF=0/CUDNN_HALF=1/" Makefile
!make

In [ ]:
import cv2
from typing import Tuple
import numpy as np
import os.path
import pandas as pd
import torch
from torch import autocast
from translate import Translator
from flask import Flask, render_template, request, redirect, jsonify
import requests
from rutube import Rutube
from bs4 import BeautifulSoup
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import os, json, random
from google.colab.patches import cv2_imshow
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from kandinsky2 import get_kandinsky2
from time import sleep
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
from google.colab import drive
drive.mount('/content/drive/')
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]

Список всех наших объектов для распознавания на видео

In [ ]:
nums = ['person\n', 'bicycle\n', 'car\n', 'motorbike\n', 'aeroplane\n', 'bus\n', 'train\n', 'truck\n', 'boat\n', 'traffic light\n', 'fire hydrant\n', 'stop sign\n', 'parking meter\n', 'bench\n', 'bird\n', 'cat\n', 'dog\n', 'horse\n', 'sheep\n', 'cow\n', 'elephant\n', 'bear\n', 'zebra\n', 'giraffe\n', 'backpack\n', 'umbrella\n', 'handbag\n', 'tie\n', 'suitcase\n', 'frisbee\n', 'skis\n', 'snowboard\n', 'sports ball\n', 'kite\n', 'baseball bat\n', 'baseball glove\n', 'skateboard\n', 'surfboard\n', 'tennis racket\n', 'bottle\n', 'wine glass\n', 'cup\n', 'fork\n', 'knife\n', 'spoon\n', 'bowl\n', 'banana\n', 'apple\n', 'sandwich\n', 'orange\n', 'broccoli\n', 'carrot\n', 'hot dog\n', 'pizza\n', 'donut\n', 'cake\n', 'chair\n', 'sofa\n', 'pottedplant\n', 'bed\n', 'diningtable\n', 'toilet\n', 'tvmonitor\n', 'laptop\n', 'mouse\n', 'remote\n', 'keyboard\n', 'cell phone\n', 'microwave\n', 'oven\n', 'toaster\n', 'sink\n', 'refrigerator\n', 'book\n', 'clock\n', 'vase\n', 'scissors\n', 'teddy bear\n', 'hair drier\n', 'toothbrush\n']

Модель Кандинского

In [ ]:
model = get_kandinsky2('cuda', task_type='text2img', model_version='2.2')

Функция для разбиения видео на кадры

In [ ]:
import os
from random import randint
import shutil


def split_video(video_path):
    file_name = video_path.rstrip('.mp4')
    os.mkdir('/content/Frames')
    video = cv2.VideoCapture(f'/content/{video_path}')
    frame_count = 0
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = total_frames // 60
    count = 0
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        if frame_count % 60 == 0:
            cv2.waitKey(1)
            cv2.imwrite(fr"/content/Frames/{count}.jpg", frame)
            count += 1
        frame_count += 1
    video.release()
    cv2.destroyAllWindows()
    return frame_interval
    # shutil.rmtree('c:\sochi\Frames')


Объект класса Translator для перевода промта с русского языка на английский

In [ ]:
translator = Translator(from_lang='ru', to_lang='en')

Получаем данные от пользователя

In [ ]:
file_name = input('Введите путь до файла: ')
# description = input('Введите описание видео: ')
wish = input('Введите свои пожелания: ')
text = input('Введите желаемый текст для картинки: ')
# models = {
#           'anime': 'hakurei/waifu-diffusion',
#           '3d': 'runwayml/stable-diffusion-v1-5',
#           'realistic': 'CompVis/stable-diffusion-v1-4'
# }
# model_id = models[input('Введите стиль картинки: realistic, anime, 3d\n')]
prompt = wish
prompt = translator.translate(prompt) + ' '

Сплитим видео на кадры

In [ ]:
frame_interval = split_video(file_name)

Функция для удаления \n с названия объекта

In [ ]:
def delete_n(arr) -> list:
  for i in range(len(arr)):
    arr[i] = arr[i][:-1]
  return arr

Получаем объекты на видео и заносим в словарь

In [ ]:
counts_dict = {}

path = '/content/Frames/'
for j in range(frame_interval):
    im = cv2.imread(f'{path}{j}.jpg')
    cfg = get_cfg()
    # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
    # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    predictor = DefaultPredictor(cfg)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    for i in list(outputs["instances"].pred_classes):
        if f'{nums[i]}' not in counts_dict.keys():
          counts_dict[f'{nums[i]}'] = 0
        counts_dict[f'{nums[i]}'] += 1
    # cv2_imshow(out.get_image()[:, :, ::-1])
    print(f'Кадр номер: {j}')
shutil.rmtree('/content/Frames')
sorted_dict = [i[0] for i in sorted(counts_dict.items()) if i[1] > 5][:5]
prompt += ' '.join(sorted_dict)

Функция для масштабирования текста на фото

In [ ]:
def choose_font_size(image_width, image_height):
  font_size = min(image_width, image_height) // 10

  return font_size

In [ ]:
prompt

Генерируем фотографию и сохраняем ее

In [ ]:
images = model.generate_text2img(
    prompt,
    h=1080,
    w=1920,
)
image = images[0]
image.save(f"/content/image.png")

In [ ]:
cv2_imshow('/content/image.png')

Наложим текст на фотографию и взглянем на результат

In [ ]:
img = Image.open('/content/image.png')
draw = ImageDraw.Draw(img)
x, y = img.size
font = ImageFont.truetype('/content/News.ttf', choose_font_size(x, y))
width = font.getlength(text)
pos = (x - width) / 2
draw.text((pos, y * 0.75), translator.translate(text), (255, 255, 255), font=font)
img.save(f'/content/sample-out.jpg')
cv2_imshow(cv2.imread(f'/content/sample-out.jpg'))